### Table of contents
1. RDD
2. DataFrame

---

#### Tips
* `SparkContext`의 instance를 만들어야 실행가능하다.
* 참고 사이트
  * https://spark.apache.org/docs/latest/index.html#
  * http://spark.apache.org/docs/latest/api/python/index.html

In [18]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("pyspark_practice").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [2]:
dir(pyspark)

['Accumulator',
 'AccumulatorParam',
 'BarrierTaskContext',
 'BarrierTaskInfo',
 'BasicProfiler',
 'Broadcast',
 'HiveContext',
 'MarshalSerializer',
 'PickleSerializer',
 'Profiler',
 'RDD',
 'RDDBarrier',
 'Row',
 'SQLContext',
 'SparkConf',
 'SparkContext',
 'SparkFiles',
 'SparkJobInfo',
 'SparkStageInfo',
 'StatusTracker',
 'StorageLevel',
 'TaskContext',
 '_NoValue',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_globals',
 'accumulators',
 'broadcast',
 'cloudpickle',
 'conf',
 'context',
 'copy_func',
 'files',
 'find_spark_home',
 'heapq3',
 'java_gateway',
 'join',
 'keyword_only',
 'profiler',
 'rdd',
 'rddsampler',
 'resultiterable',
 'serializers',
 'shuffle',
 'since',
 'sql',
 'statcounter',
 'status',
 'storagelevel',
 'taskcontext',
 'traceback_utils',
 'types',
 'util',
 'version',
 'wraps']

### 1. RDD

In [20]:
import json

raw = ['{"name":"Drake"}','{"name":"Travis Scott"}','{"name":"Post Malone"}']
for i in raw:
    json.loads(i)

rdd = sc.parallelize(raw)
print(rdd.count())

rdd.map(json.loads).map(lambda entry:entry['name']).collect()

3


['Drake', 'Travis Scott', 'Post Malone']

### 2. DataFrame

In [ ]:
## SQL에서 읽어오기
#raw = sqlContext.read.format("parquet").load("{path}")
#df = raw.toDF('id', 'artist_id', 'name', 'external_url')

df = spark.read.json("examples/src/main/resources/people.json")
df.printSchema()
df.show()

In [ ]:
## Select subset columns
df_subset = df.select(df["id"], df["danceability"])

In [ ]:
## Filter Rows
df_subset = df_subset.filter(df_subset['danceability']>=0.30)

In [ ]:
## Aggregate
import pyspark.sql.functions as F

df_subset = df_subset.agg(F.avg(df_subset['danceability']), F.max(df_subset['danceability']))
df_subset = df_subset.agg(F.avg(df_subset['danceability']).alias('avg_danceability'))

In [ ]:
## User define funtion
from pyspark.sql.functions import udf
from pyspark.sql.types import *

udf1 = udf(lambda e:e.upper())

@udf(returnType=BooleanType())
def udf2(e):
    if e >= 0.06:
        return True
    else:
        return False
    
df_filtered = df.filter(udf2(df['danceability']))

In [ ]:
## Join
joined = artists.join(df, df['artist_id']==artists['id'])